### Delete Existing Resources

In [1]:
%%time
!python scripts/delete_resources.py

ERROR:root:Cluster 'dwhCluster' does not exist.
ERROR:root:Another error occurred: CusterProps must not be None. Skipping Ingress revocation.
ERROR:root:Cluster 'dwhCluster' does not exist. Skipping cluster deletion.
ERROR:root:IAM role 'dwhRole' does not exist. Skipping IAM role deletion.
CPU times: user 8.95 ms, sys: 11 ms, total: 20 ms
Wall time: 1.62 s


### Create Required Resources
This includes the IAM role, RedShift cluster, and ingress rules.

In [2]:
%%time
!python scripts/create_resources.py

INFO:root:IAM Role Created: ARN arn:aws:iam::736720720705:role/dwhRole
INFO:root:Cluster is creating...2.384185791015625e-07 seconds elapsed.
INFO:root:Cluster is creating...10.238889217376709 seconds elapsed.
INFO:root:Cluster is creating...20.997263431549072 seconds elapsed.
INFO:root:Cluster is creating...31.69753861427307 seconds elapsed.
INFO:root:Cluster is creating...42.36496829986572 seconds elapsed.
INFO:root:Cluster is creating...53.10138392448425 seconds elapsed.
INFO:root:Cluster is creating...63.818397998809814 seconds elapsed.
INFO:root:Cluster is creating...74.51712822914124 seconds elapsed.
CPU times: user 671 ms, sys: 239 ms, total: 911 ms
Wall time: 1min 29s


### Create Tables
Staging tables will first be created, followed by fact and dimension tables.

In [5]:
%%time
!python scripts/create_tables.py

CPU times: user 45 ms, sys: 24.5 ms, total: 69.6 ms
Wall time: 6.27 s


**ERD**

Outlined below is the entity relationship diagram for the generated tables:

![erd](images/erd.png)

The fact and dimension tables are arranged in a star schema, while the staging tables are isolated.

### Load Data into the Staging Tables

In [7]:
%%time
!python scripts/etl.py load

INFO:root:Loading staging data.
INFO:root:Staging tables loaded.
CPU times: user 37.9 s, sys: 12.3 s, total: 50.2 s
Wall time: 1h 54min 11s
